## Objectives

* Request to the SpaceX API
* Clean the requested data

In [1]:
# Requests allows us to make HTTP requests which we will use to get data from an API
import requests
# Pandas is a software library written for the Python programming language for data manipulation and analysis.
import pandas as pd
# NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import numpy as np
# Datetime is a library that allows us to represent dates
import datetime

# Setting this option will print all collumns of a dataframe
pd.set_option('display.max_columns', None)
# Setting this option will print all of the data in a feature
pd.set_option('display.max_colwidth', None)

In [2]:
# Takes the dataset and uses the rocket column to call the API and append the data to the list
def getBoosterVersion(data):
    for x in data['rocket']:
        response = requests.get("https://api.spacexdata.com/v4/rockets/"+str(x)).json()
        BoosterVersion.append(response['name'])

In [3]:
# Takes the dataset and uses the launchpad column to call the API and append the data to the list
def getLaunchSite(data):
    for x in data['launchpad']:
        response = requests.get("https://api.spacexdata.com/v4/launchpads/"+str(x)).json()
        Longitude.append(response['longitude'])
        Latitude.append(response['latitude'])
        LaunchSite.append(response['name'])

In [4]:
# Takes the dataset and uses the payloads column to call the API and append the data to the lists
def getPayloadData(data):
    for load in data['payloads']:
        response = requests.get("https://api.spacexdata.com/v4/payloads/"+load).json()
        PayloadMass.append(response['mass_kg'])
        Orbit.append(response['orbit'])

In [5]:
# Takes the dataset and uses the cores column to call the API and append the data to the lists
def getCoreData(data):
    for core in data['cores']:
            if core['core'] != None:
                response = requests.get("https://api.spacexdata.com/v4/cores/"+core['core']).json()
                Block.append(response['block'])
                ReusedCount.append(response['reuse_count'])
                Serial.append(response['serial'])
            else:
                Block.append(None)
                ReusedCount.append(None)
                Serial.append(None)
            Outcome.append(str(core['landing_success'])+' '+str(core['landing_type']))
            Flights.append(core['flight'])
            GridFins.append(core['gridfins'])
            Reused.append(core['reused'])
            Legs.append(core['legs'])
            LandingPad.append(core['landpad'])

In [6]:
spacex_url="https://api.spacexdata.com/v4/launches/past"

In [7]:
response = requests.get(spacex_url)

In [10]:
# decode response content as json
static_json_df = response.json()

In [11]:
# apply json_normalize
data = pd.json_normalize(static_json_df)

In [13]:
# Get the head of the dataframe
data.head(1)

,static_fire_date_utc,static_fire_date_unix,net,window,rocket,success,failures,details,crew,ships,capsules,payloads,launchpad,flight_number,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,id,fairings.reused,fairings.recovery_attempt,fairings.recovered,fairings.ships,links.patch.small,links.patch.large,links.reddit.campaign,links.reddit.launch,links.reddit.media,links.reddit.recovery,links.flickr.small,links.flickr.original,links.presskit,links.webcast,links.youtube_id,links.article,links.wikipedia,fairings
0,2006-03-17T00:00:00.000Z,1.142554e+09,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 33, 'altitude': None, 'reason': 'merlin engine failure'}]",Engine failure at 33 seconds and loss of vehicle,[],[],[],[5eb0e4b5b6c3bb0006eeb1e1],5e9e4502f5090995de566f86,1,FalconSat,2006-03-24T22:30:00.000Z,1143239400,2006-03-25T10:30:00+12:00,hour,False,"[{'core': '5e9e289df35918033d3b2623', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",True,False,None,5eb87cd9ffd86e000604b32a,False,False,False,[],https://images2.imgbox.com/3c/0e/T8iJcSN3_o.png,https://images2.imgbox.com/40/e3/GypSkayF_o.png,None,None,None,None,[],[],None,https://www.youtube.com/watch?v=0a_00nJ_Y88,0a_00nJ_Y88,https://www.space.com/2196-spacex-inaugural-falcon-1-rocket-lost-launch.html,https://en.wikipedia.org/wiki/DemoSat,NaN


In [14]:
# Lets take a subset of our dataframe keeping only the features we want and the flight number, and date_utc.
data = data[['rocket', 'payloads', 'launchpad', 'cores', 'flight_number', 'date_utc']]

# We will remove rows with multiple cores because those are falcon rockets with 2 extra rocket boosters and rows that have multiple payloads in a single rocket.
data = data[data['cores'].map(len)==1]
data = data[data['payloads'].map(len)==1]

# Since payloads and cores are lists of size 1 we will also extract the single value in the list and replace the feature.
data['cores'] = data['cores'].map(lambda x : x[0])
data['payloads'] = data['payloads'].map(lambda x : x[0])

# We also want to convert the date_utc to a datetime datatype and then extracting the date leaving the time
data['date'] = pd.to_datetime(data['date_utc']).dt.date

# Using the date we will restrict the dates of the launches
data = data[data['date'] <= datetime.date(2020, 11, 13)]

In [15]:
#Global variables 
BoosterVersion = []
PayloadMass = []
Orbit = []
LaunchSite = []
Outcome = []
Flights = []
GridFins = []
Reused = []
Legs = []
LandingPad = []
Block = []
ReusedCount = []
Serial = []
Longitude = []
Latitude = []

In [16]:
# Call getBoosterVersion
getBoosterVersion(data)

In [17]:
BoosterVersion[0:5]

['Falcon 1', 'Falcon 1', 'Falcon 1', 'Falcon 1', 'Falcon 9']

In [19]:
# Call getPayloadData
getPayloadData(data)

In [20]:
# Call getCoreData
getCoreData(data)

In [21]:
launch_dict = {'FlightNumber': list(data['flight_number']),
'Date': list(data['date']),
'BoosterVersion':BoosterVersion,
'PayloadMass':PayloadMass,
'Orbit':Orbit,
'LaunchSite':LaunchSite,
'Outcome':Outcome,
'Flights':Flights,
'GridFins':GridFins,
'Reused':Reused,
'Legs':Legs,
'LandingPad':LandingPad,
'Block':Block,
'ReusedCount':ReusedCount,
'Serial':Serial,
'Longitude': Longitude,
'Latitude': Latitude}

In [22]:
# Create a data from launch_dict
launch_df = pd.DataFrame([launch_dict])

In [23]:
# Show the head of the dataframe
launch_df.head(2)

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,"[1, 2, 4, 5, 6, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 57, 58, 59, 60, 61, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 76, 78, 79, 80, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 100, 101, 102, 103, 104, 105, 106]","[2006-03-24, 2007-03-21, 2008-09-28, 2009-07-13, 2010-06-04, 2012-05-22, 2013-03-01, 2013-09-29, 2013-12-03, 2014-01-06, 2014-04-18, 2014-07-14, 2014-08-05, 2014-09-07, 2014-09-21, 2015-01-10, 2015-02-11, 2015-04-14, 2015-04-27, 2015-06-28, 2015-12-22, 2016-01-17, 2016-03-04, 2016-04-08, 2016-05-06, 2016-05-27, 2016-07-18, 2016-08-14, 2016-09-01, 2017-01-14, 2017-02-19, 2017-03-16, 2017-03-30, 2017-05-01, 2017-05-15, 2017-06-03, 2017-06-23, 2017-06-25, 2017-07-05, 2017-08-14, 2017-08-24, 2017-09-07, 2017-10-09, 2017-10-11, 2017-10-30, 2017-12-15, 2017-12-23, 2018-01-08, 2018-01-31, 2018-03-06, 2018-03-30, 2018-04-02, 2018-04-18, 2018-05-11, 2018-06-04, 2018-06-29, 2018-07-22, 2018-07-25, 2018-08-07, 2018-09-10, 2018-10-08, 2018-11-15, 2018-12-03, 2018-12-05, 2018-12-23, 2019-01-11, 2019-03-02, 2019-05-04, 2019-05-24, 2019-06-12, 2019-07-25, 2019-08-06, 2019-11-11, 2019-12-05, 2019-12-17, 2020-01-07, 2020-01-19, 2020-01-29, 2020-02-17, 2020-03-07, 2020-03-18, 2020-04-22, 2020-05-30, 2020-06-04, 2020-06-13, 2020-06-30, 2020-07-20, 2020-08-18, 2020-08-30, 2020-09-03, 2020-10-06, 2020-10-18, 2020-10-24, 2020-11-05]","[Falcon 1, Falcon 1, Falcon 1, Falcon 1, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9]","[20, None, 165, 200, None, 525, 677, 500, 3170, 3325, 2296, 1316, 4535, 4428, 2216, 2395, 570, 1898, 4707, 2477, 2034, 553, 5271, 3136, 4696, 3100, 2257, 4600, 5500, 9600, 2490, 5600, 5300, None, 6070, 2708, 3669, 9600, 6761, 2910, 475, 4990, 9600, 5200, 3700, 2205, 9600, None, 4230, 6092, 9600, 2760, 350, 3750, 5383.85, 2410, 7076, 9600, 5800, 7060, 2800, 3000, 4000, 2573, 4400, 9600, 12259, 2482, 13200, 1425, 2227.7, 6500, 15600, 5000, 6800, 15600, None, 15600, 15600, 1977, 15600, 15600, 9525, 15600, 15600, 3880, None, 15600, 1600, 15600, 15600, 15600, 15600, 3681]","[LEO, LEO, LEO, LEO, LEO, LEO, ISS, PO, GTO, GTO, ISS, LEO, GTO, GTO, ISS, ISS, ES-L1, ISS, GTO, ISS, LEO, PO, GTO, ISS, GTO, GTO, ISS, GTO, GTO, PO, ISS, GTO, GTO, LEO, GTO, ISS, GTO, PO, GTO, ISS, SSO, LEO, PO, GTO, GTO, ISS, PO, LEO, GTO, GTO, PO, ISS, HEO, GTO, GTO, ISS, GTO, PO, GTO, GTO, SSO, GTO, SSO, ISS, MEO, PO, ISS, ISS, VLEO, SSO, ISS, GTO, VLEO, ISS, GTO, VLEO, SO, VLEO, VLEO, ISS, VLEO, VLEO, ISS, VLEO, VLEO, MEO, GEO, VLEO, SSO, VLEO, VLEO, VLEO, VLEO, MEO]",[],"[None None, None None, None None, None None, None None, None None, None None, False Ocean, None None, None None, True Ocean, True Ocean, None None, None None, False Ocean, False ASDS, True Ocean, False ASDS, None None, None ASDS, True RTLS, False ASDS, False ASDS, True ASDS, True ASDS, True ASDS, True RTLS, True ASDS, None ASDS,

In [24]:
# Hint data['BoosterVersion']!='Falcon 1'
data_falcon9 = launch_df[launch_df['BoosterVersion'] != 'Falcon 1']
data_falcon9

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,"[1, 2, 4, 5, 6, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 57, 58, 59, 60, 61, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 76, 78, 79, 80, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 100, 101, 102, 103, 104, 105, 106]","[2006-03-24, 2007-03-21, 2008-09-28, 2009-07-13, 2010-06-04, 2012-05-22, 2013-03-01, 2013-09-29, 2013-12-03, 2014-01-06, 2014-04-18, 2014-07-14, 2014-08-05, 2014-09-07, 2014-09-21, 2015-01-10, 2015-02-11, 2015-04-14, 2015-04-27, 2015-06-28, 2015-12-22, 2016-01-17, 2016-03-04, 2016-04-08, 2016-05-06, 2016-05-27, 2016-07-18, 2016-08-14, 2016-09-01, 2017-01-14, 2017-02-19, 2017-03-16, 2017-03-30, 2017-05-01, 2017-05-15, 2017-06-03, 2017-06-23, 2017-06-25, 2017-07-05, 2017-08-14, 2017-08-24, 2017-09-07, 2017-10-09, 2017-10-11, 2017-10-30, 2017-12-15, 2017-12-23, 2018-01-08, 2018-01-31, 2018-03-06, 2018-03-30, 2018-04-02, 2018-04-18, 2018-05-11, 2018-06-04, 2018-06-29, 2018-07-22, 2018-07-25, 2018-08-07, 2018-09-10, 2018-10-08, 2018-11-15, 2018-12-03, 2018-12-05, 2018-12-23, 2019-01-11, 2019-03-02, 2019-05-04, 2019-05-24, 2019-06-12, 2019-07-25, 2019-08-06, 2019-11-11, 2019-12-05, 2019-12-17, 2020-01-07, 2020-01-19, 2020-01-29, 2020-02-17, 2020-03-07, 2020-03-18, 2020-04-22, 2020-05-30, 2020-06-04, 2020-06-13, 2020-06-30, 2020-07-20, 2020-08-18, 2020-08-30, 2020-09-03, 2020-10-06, 2020-10-18, 2020-10-24, 2020-11-05]","[Falcon 1, Falcon 1, Falcon 1, Falcon 1, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9]","[20, None, 165, 200, None, 525, 677, 500, 3170, 3325, 2296, 1316, 4535, 4428, 2216, 2395, 570, 1898, 4707, 2477, 2034, 553, 5271, 3136, 4696, 3100, 2257, 4600, 5500, 9600, 2490, 5600, 5300, None, 6070, 2708, 3669, 9600, 6761, 2910, 475, 4990, 9600, 5200, 3700, 2205, 9600, None, 4230, 6092, 9600, 2760, 350, 3750, 5383.85, 2410, 7076, 9600, 5800, 7060, 2800, 3000, 4000, 2573, 4400, 9600, 12259, 2482, 13200, 1425, 2227.7, 6500, 15600, 5000, 6800, 15600, None, 15600, 15600, 1977, 15600, 15600, 9525, 15600, 15600, 3880, None, 15600, 1600, 15600, 15600, 15600, 15600, 3681]","[LEO, LEO, LEO, LEO, LEO, LEO, ISS, PO, GTO, GTO, ISS, LEO, GTO, GTO, ISS, ISS, ES-L1, ISS, GTO, ISS, LEO, PO, GTO, ISS, GTO, GTO, ISS, GTO, GTO, PO, ISS, GTO, GTO, LEO, GTO, ISS, GTO, PO, GTO, ISS, SSO, LEO, PO, GTO, GTO, ISS, PO, LEO, GTO, GTO, PO, ISS, HEO, GTO, GTO, ISS, GTO, PO, GTO, GTO, SSO, GTO, SSO, ISS, MEO, PO, ISS, ISS, VLEO, SSO, ISS, GTO, VLEO, ISS, GTO, VLEO, SO, VLEO, VLEO, ISS, VLEO, VLEO, ISS, VLEO, VLEO, MEO, GEO, VLEO, SSO, VLEO, VLEO, VLEO, VLEO, MEO]",[],"[None None, None None, None None, None None, None None, None None, None None, False Ocean, None None, None None, True Ocean, True Ocean, None None, None None, False Ocean, False ASDS, True Ocean, False ASDS, None None, None ASDS, True RTLS, False ASDS, False ASDS, True ASDS, True ASDS, True ASDS, True RTLS, True ASDS, None ASDS,

In [25]:
data_falcon9.loc[:,'FlightNumber'] = list(range(1, data_falcon9.shape[0]+1))
data_falcon9

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1,"[2006-03-24, 2007-03-21, 2008-09-28, 2009-07-13, 2010-06-04, 2012-05-22, 2013-03-01, 2013-09-29, 2013-12-03, 2014-01-06, 2014-04-18, 2014-07-14, 2014-08-05, 2014-09-07, 2014-09-21, 2015-01-10, 2015-02-11, 2015-04-14, 2015-04-27, 2015-06-28, 2015-12-22, 2016-01-17, 2016-03-04, 2016-04-08, 2016-05-06, 2016-05-27, 2016-07-18, 2016-08-14, 2016-09-01, 2017-01-14, 2017-02-19, 2017-03-16, 2017-03-30, 2017-05-01, 2017-05-15, 2017-06-03, 2017-06-23, 2017-06-25, 2017-07-05, 2017-08-14, 2017-08-24, 2017-09-07, 2017-10-09, 2017-10-11, 2017-10-30, 2017-12-15, 2017-12-23, 2018-01-08, 2018-01-31, 2018-03-06, 2018-03-30, 2018-04-02, 2018-04-18, 2018-05-11, 2018-06-04, 2018-06-29, 2018-07-22, 2018-07-25, 2018-08-07, 2018-09-10, 2018-10-08, 2018-11-15, 2018-12-03, 2018-12-05, 2018-12-23, 2019-01-11, 2019-03-02, 2019-05-04, 2019-05-24, 2019-06-12, 2019-07-25, 2019-08-06, 2019-11-11, 2019-12-05, 2019-12-17, 2020-01-07, 2020-01-19, 2020-01-29, 2020-02-17, 2020-03-07, 2020-03-18, 2020-04-22, 2020-05-30, 2020-06-04, 2020-06-13, 2020-06-30, 2020-07-20, 2020-08-18, 2020-08-30, 2020-09-03, 2020-10-06, 2020-10-18, 2020-10-24, 2020-11-05]","[Falcon 1, Falcon 1, Falcon 1, Falcon 1, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9]","[20, None, 165, 200, None, 525, 677, 500, 3170, 3325, 2296, 1316, 4535, 4428, 2216, 2395, 570, 1898, 4707, 2477, 2034, 553, 5271, 3136, 4696, 3100, 2257, 4600, 5500, 9600, 2490, 5600, 5300, None, 6070, 2708, 3669, 9600, 6761, 2910, 475, 4990, 9600, 5200, 3700, 2205, 9600, None, 4230, 6092, 9600, 2760, 350, 3750, 5383.85, 2410, 7076, 9600, 5800, 7060, 2800, 3000, 4000, 2573, 4400, 9600, 12259, 2482, 13200, 1425, 2227.7, 6500, 15600, 5000, 6800, 15600, None, 15600, 15600, 1977, 15600, 15600, 9525, 15600, 15600, 3880, None, 15600, 1600, 15600, 15600, 15600, 15600, 3681]","[LEO, LEO, LEO, LEO, LEO, LEO, ISS, PO, GTO, GTO, ISS, LEO, GTO, GTO, ISS, ISS, ES-L1, ISS, GTO, ISS, LEO, PO, GTO, ISS, GTO, GTO, ISS, GTO, GTO, PO, ISS, GTO, GTO, LEO, GTO, ISS, GTO, PO, GTO, ISS, SSO, LEO, PO, GTO, GTO, ISS, PO, LEO, GTO, GTO, PO, ISS, HEO, GTO, GTO, ISS, GTO, PO, GTO, GTO, SSO, GTO, SSO, ISS, MEO, PO, ISS, ISS, VLEO, SSO, ISS, GTO, VLEO, ISS, GTO, VLEO, SO, VLEO, VLEO, ISS, VLEO, VLEO, ISS, VLEO, VLEO, MEO, GEO, VLEO, SSO, VLEO, VLEO, VLEO, VLEO, MEO]",[],"[None None, None None, None None, None None, None None, None None, None None, False Ocean, None None, None None, True Ocean, True Ocean, None None, None None, False Ocean, False ASDS, True Ocean, False ASDS, None None, None ASDS, True RTLS, False ASDS, False ASDS, True ASDS, True ASDS, True ASDS, True RTLS, True ASDS, None ASDS, True ASDS, True RTLS, None None, True ASDS, True RTLS, None None, True RTLS, True ASDS, True ASDS, None None, True RTLS, True ASDS, True RTLS, True ASDS, True ASDS, True ASDS, True RTLS, True Ocean, True RTLS, True Ocean, None None, None None, None None, True ASDS, True ASDS, None None, None None, True ASDS, True ASDS, True ASDS, True ASDS, True RTLS, True ASDS, True ASDS, F

In [26]:
data_falcon9.isnull().sum()

FlightNumber      0
Date              0
BoosterVersion    0
PayloadMass       0
Orbit             0
LaunchSite        0
Outcome           0
Flights           0
GridFins          0
Reused            0
Legs              0
LandingPad        0
Block             0
ReusedCount       0
Serial            0
Longitude         0
Latitude          0
dtype: int64

In [27]:
# Calculate the mean value of PayloadMass column
PayloadMass = pd.DataFrame(data_falcon9['PayloadMass'].values.tolist()).mean(1)
print(PayloadMass)  

0    5919.165341
dtype: float64


In [28]:
rows = data_falcon9['PayloadMass'].values.tolist()[0]

df_rows = pd.DataFrame(rows)
df_rows = df_rows.replace(np.nan, PayloadMass)

data_falcon9['PayloadMass'][0] = df_rows.values
data_falcon9

/tmp/wsuser/ipykernel_154/1046548779.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_falcon9['PayloadMass'][0] = df_rows.values


,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1,"[2006-03-24, 2007-03-21, 2008-09-28, 2009-07-13, 2010-06-04, 2012-05-22, 2013-03-01, 2013-09-29, 2013-12-03, 2014-01-06, 2014-04-18, 2014-07-14, 2014-08-05, 2014-09-07, 2014-09-21, 2015-01-10, 2015-02-11, 2015-04-14, 2015-04-27, 2015-06-28, 2015-12-22, 2016-01-17, 2016-03-04, 2016-04-08, 2016-05-06, 2016-05-27, 2016-07-18, 2016-08-14, 2016-09-01, 2017-01-14, 2017-02-19, 2017-03-16, 2017-03-30, 2017-05-01, 2017-05-15, 2017-06-03, 2017-06-23, 2017-06-25, 2017-07-05, 2017-08-14, 2017-08-24, 2017-09-07, 2017-10-09, 2017-10-11, 2017-10-30, 2017-12-15, 2017-12-23, 2018-01-08, 2018-01-31, 2018-03-06, 2018-03-30, 2018-04-02, 2018-04-18, 2018-05-11, 2018-06-04, 2018-06-29, 2018-07-22, 2018-07-25, 2018-08-07, 2018-09-10, 2018-10-08, 2018-11-15, 2018-12-03, 2018-12-05, 2018-12-23, 2019-01-11, 2019-03-02, 2019-05-04, 2019-05-24, 2019-06-12, 2019-07-25, 2019-08-06, 2019-11-11, 2019-12-05, 2019-12-17, 2020-01-07, 2020-01-19, 2020-01-29, 2020-02-17, 2020-03-07, 2020-03-18, 2020-04-22, 2020-05-30, 2020-06-04, 2020-06-13, 2020-06-30, 2020-07-20, 2020-08-18, 2020-08-30, 2020-09-03, 2020-10-06, 2020-10-18, 2020-10-24, 2020-11-05]","[Falcon 1, Falcon 1, Falcon 1, Falcon 1, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9, Falcon 9]","[[20.0], [5919.16534090909], [165.0], [200.0], [5919.16534090909], [525.0], [677.0], [500.0], [3170.0], [3325.0], [2296.0], [1316.0], [4535.0], [4428.0], [2216.0], [2395.0], [570.0], [1898.0], [4707.0], [2477.0], [2034.0], [553.0], [5271.0], [3136.0], [4696.0], [3100.0], [2257.0], [4600.0], [5500.0], [9600.0], [2490.0], [5600.0], [5300.0], [5919.16534090909], [6070.0], [2708.0], [3669.0], [9600.0], [6761.0], [2910.0], [475.0], [4990.0], [9600.0], [5200.0], [3700.0], [2205.0], [9600.0], [5919.16534090909], [4230.0], [6092.0], [9600.0], [2760.0], [350.0], [3750.0], [5383.85], [2410.0], [7076.0], [9600.0], [5800.0], [7060.0], [2800.0], [3000.0], [4000.0], [2573.0], [4400.0], [9600.0], [12259.0], [2482.0], [13200.0], [1425.0], [2227.7], [6500.0], [15600.0], [5000.0], [6800.0], [15600.0], [5919.16534090909], [15600.0], [15600.0], [1977.0], [15600.0], [15600.0], [9525.0], [15600.0], [15600.0], [3880.0], [5919.16534090909], [15600.0], [1600.0], [15600.0], [15600.0], [15600.0], [15600.0], [3681.0]]","[LEO, LEO, LEO, LEO, LEO, LEO, ISS, PO, GTO, GTO, ISS, LEO, GTO, GTO, ISS, ISS, ES-L1, ISS, GTO, ISS, LEO, PO, GTO, ISS, GTO, GTO, ISS, GTO, GTO, PO, ISS, GTO, GTO, LEO, GTO, ISS, GTO, PO, GTO, ISS, SSO, LEO, PO, GTO, GTO, ISS, PO, LEO, GTO, GTO, PO, ISS, HEO, GTO, GTO, ISS, GTO, PO, GTO, GTO, SSO, GTO, SSO, ISS, MEO, PO, ISS, ISS, VLEO, SSO, ISS, GTO, VLEO, ISS, GTO, VLEO, SO, VLEO, VLEO, ISS, VLEO, VLEO, ISS, VLEO, VLEO, MEO, GEO, VLEO, SSO, VLEO, VLEO, VLEO, VLEO, MEO]",[],"[None None, None None, None None, None None, None None, None None, None None, False Ocean, None None, None None, True Ocean, True Ocean, None None, None None, False Ocean, False ASDS, True Ocean, False ASDS, None None, None ASDS, True RTLS, False ASDS, False ASDS, True ASDS, 

In [29]:
# missing values in `landing_rows`
landing_rows = data_falcon9['LandingPad'].values.tolist()[0]
landing_rows = pd.DataFrame(landing_rows)
landing_rows.isnull().sum()

0    30
dtype: int64

In [30]:
# save and export new dataset
data_falcon9.to_csv('dataset_part_1.csv', index=False)